In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

In [0]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [0]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [0]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [0]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/My Drive/Colab Notebooks/HumanActivityRecognition.zip (Unzipped Files)/HAR/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [0]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'/content/drive/My Drive/Colab Notebooks/HumanActivityRecognition.zip (Unzipped Files)/HAR/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [0]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [0]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [0]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [11]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [0]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [0]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [0]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [15]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [16]:
timesteps = 128
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)
input_shape = (timesteps, input_dim)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


**LSTM Model 2**

We tried several different models with varying number of layers an different dropout rates. The best accuracy we achived is 93.8% with thebelow model.

In [0]:
from keras.layers import BatchNormalization
# Initiliazing the sequential model
model2 = Sequential()
# Configuring the parameters
model2.add(LSTM(128, return_sequences=True, input_shape=(timesteps, input_dim), kernel_initializer='glorot_normal'))
# Adding a dropout layer
model2.add(Dropout(0.2))
model2.add(BatchNormalization())
model2.add(LSTM(64))
# Adding a dropout layer
model2.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model2.add(Dense(n_classes, activation='sigmoid'))
model2.summary()

W0831 12:02:43.248532 139997190457216 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0831 12:02:43.254620 139997190457216 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0831 12:02:43.269654 139997190457216 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4479: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0831 12:02:43.571439 139997190457216 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0831 12:02

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128, 128)          70656     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128)          512       
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 390       
Total params: 120,966
Trainable params: 120,710
Non-trainable params: 256
______________________________________________

In [0]:
# Compiling the model
model2.compile(loss='categorical_crossentropy
              optimizer='rmsprop',
              metrics=['accuracy'])

from keras.callbacks import ModelCheckpoint

filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [0]:
# Training the model
model2.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=epochs, callbacks=callbacks_list)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 236s 32ms/step - loss: 0.2719 - acc: 0.9102 - val_loss: 0.2652 - val_acc: 0.9060

Epoch 00001: val_acc improved from -inf to 0.90601, saving model to weights-improvement-01-0.91.hdf5
Epoch 2/30
7352/7352 [==============================] - 235s 32ms/step - loss: 0.1989 - acc: 0.9304 - val_loss: 0.2596 - val_acc: 0.8972

Epoch 00002: val_acc did not improve from 0.90601
Epoch 3/30
7352/7352 [==============================] - 234s 32ms/step - loss: 0.1877 - acc: 0.9289 - val_loss: 0.1934 - val_acc: 0.9169

Epoch 00003: val_acc improved from 0.90601 to 0.91686, saving model to weights-improvement-03-0.92.hdf5
Epoch 4/30
7352/7352 [==============================] - 236s 32ms/step - loss: 0.1766 - acc: 0.9331 - val_loss: 0.1692 - val_acc: 0.9318

Epoch 00004: val_acc improved from 0.91686 to 0.93180, saving model to weights-improvement-04-0.93.hdf5
Epoch 5/30
7352/7352 [===================

In [0]:
## Confusion Matrix
print(confusion_matrix(Y_test, model2.predict(X_test)))

Pred                STANDING  WALKING
True                                 
LAYING                     0      537
SITTING                    0      491
STANDING                   0      532
WALKING                    0      496
WALKING_DOWNSTAIRS         0      420
WALKING_UPSTAIRS           1      470


In [0]:
score = model2.evaluate(X_test, Y_test)

2947/2947 [==============================] - 17s 6ms/step


In [0]:
score


[1.7911514966496784, 0.168306752629793]

In [0]:
from keras.layers import BatchNormalization
# Initiliazing the sequential model
model2 = Sequential()
# Configuring the parameters
model2.add(LSTM(128, return_sequences=True, input_shape=(timesteps, input_dim), kernel_initializer='glorot_normal'))
# Adding a dropout layer
model2.add(Dropout(0.2))
model2.add(BatchNormalization())
model2.add(LSTM(64))
# Adding a dropout layer
model2.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model2.add(Dense(n_classes, activation='sigmoid'))
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128, 128)          70656     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128, 128)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 128)          512       
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
Total params: 120,966
Trainable params: 120,710
Non-trainable params: 256
______________________________________________

In [0]:
# Compiling the model
model2.load_weights("/content/weights-improvement-16-0.94.hdf5")
model2.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

from keras.callbacks import ModelCheckpoint

filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [0]:
scores = model2.evaluate(X_test, Y_test)
print("%s: %.2f%%" % (model2.metrics_names[1], scores[1]*100))

2947/2947 [==============================] - 18s 6ms/step
acc: 93.86%


In [0]:
## Confusion Matrix
print(confusion_matrix(Y_test, model2.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 537        0  ...                   0                 0
SITTING                  0      372  ...                   0                 1
STANDING                 0       46  ...                   0                 0
WALKING                  0        0  ...                   2                 0
WALKING_DOWNSTAIRS       0        0  ...                 420                 0
WALKING_UPSTAIRS         0        0  ...                   3               459

[6 rows x 6 columns]


**Conclusions**

In [0]:
from prettytable import PrettyTable
    
x = PrettyTable()

x.field_names = ["Model Name","Number of LSTM Layers", "Dropout Implemented", "Batch Normalization implemented","Epochs Run","Initializer used","Optimizer", "Accuracy Achieved"]

x.add_row(["Model 1", 1, 0.5,"No", 30,"none","Sigmoid", 0.908 ])
x.add_row(["Model with Zero Initializer", 1, 0.5,"NO","Zero", "Sigmoid", 30,0.892 ])
x.add_row(["Model with Glorot Normal Initializer", 1, 0.5,"NO","Glorot Normal", "Sigmoid", 30,0.91 ])
x.add_row(["LSTM Model 1", 2, "Yes" ,"NO","Glorot Normal", "Sigmoid", 30,0.896 ])
x.add_row(["LSTM Model 2", 2, "Yes" ,"Yes","Glorot Normal", "Sigmoid", 25 ,0.938 ])
x.add_row(["LSTM Model with RELU with Batch Normaliztion", 2, "Yes" ,"YES","Glorot Normal", "RELU", 30,0.921 ])

print(x)

+----------------------------------------------+-----------------------+---------------------+---------------------------------+---------------+------------------+-----------+-------------------+
|                  Model Name                  | Number of LSTM Layers | Dropout Implemented | Batch Normalization implemented |   Epochs Run  | Initializer used | Optimizer | Accuracy Achieved |
+----------------------------------------------+-----------------------+---------------------+---------------------------------+---------------+------------------+-----------+-------------------+
|                   Model 1                    |           1           |         0.5         |                No               |       30      |       none       |  Sigmoid  |       0.908       |
|         Model with Zero Initializer          |           1           |         0.5         |                NO               |      Zero     |     Sigmoid      |     30    |       0.892       |
|     Model with Glo

1) Initially tried using GridsearchCV to optimize parameters but since it was taking too long, we tried different parameters.
2) We used different combinations of umber of hidden layers, optimizers etc.
3) LSTM model 2 performed the best for us which uses batch normalization with sigmoid optimizer. We got an accuracy of 93.8 on 25 epochs after which it was overfitting.

**Updated Assignment**

In [0]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, Softmax
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D, TimeDistributed
from keras import backend as K

In [18]:
from keras.layers import BatchNormalization
# Initiliazing the sequential model
model1 = Sequential()
# Configuring the parameters
model1.add(LSTM(128, return_sequences=True, input_shape=(timesteps, input_dim), kernel_initializer='he_uniform'))
# Adding a dropout layer
model1.add(Dropout(0.2))
model1.add(LSTM(64, return_sequences=True))
model1.add(Conv1D(32, kernel_size=(3), activation='relu'))
model1.add(Dropout(0.5))
model1.add(Conv1D(64, kernel_size=(3) , activation='relu'))
model1.add(MaxPooling1D(pool_size=2))
model1.add(Flatten())
model1.add(Dense(50, activation='relu'))

model1.add(Dense(6, activation='softmax'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [0]:
model1.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

from keras.callbacks import ModelCheckpoint

filepath="/content/drive/My Drive/Colab Notebooks/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [89]:

history=model1.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=epochs, callbacks=callbacks_list, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 246s 33ms/step - loss: 0.3965 - acc: 0.8366 - val_loss: 0.3445 - val_acc: 0.8700

Epoch 00001: val_acc improved from -inf to 0.87004, saving model to /content/drive/My Drive/Colab Notebooks/weights-improvement-01-0.87.hdf5
Epoch 2/30
7352/7352 [==============================] - 230s 31ms/step - loss: 0.1508 - acc: 0.9392 - val_loss: 0.4574 - val_acc: 0.8938

Epoch 00002: val_acc improved from 0.87004 to 0.89379, saving model to /content/drive/My Drive/Colab Notebooks/weights-improvement-02-0.89.hdf5
Epoch 3/30
7352/7352 [==============================] - 228s 31ms/step - loss: 0.1408 - acc: 0.9445 - val_loss: 0.3957 - val_acc: 0.8897

Epoch 00003: val_acc did not improve from 0.89379
Epoch 4/30
7352/7352 [==============================] - 228s 31ms/step - loss: 0.1148 - acc: 0.9483 - val_los

In [0]:
score = model1.evaluate(X_test, Y_test)
score

**Lets try the best performing weights from model 1**

In [0]:
model1.load_weights("/content/drive/My Drive/Colab Notebooks/weights-improvement-14-0.94.hdf5")
model1.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
scores = model1.evaluate(X_test, Y_test, verbose=0)

In [26]:
print("%s: %.2f%%" % (model1.metrics_names[1], scores[1]*100))

acc: 93.82%


**Model 2**

In [0]:
from keras.layers import BatchNormalization
# Initiliazing the sequential model
model2 = Sequential()
# Configuring the parameters
model2.add(LSTM(96, return_sequences=True, input_shape=(timesteps, input_dim), kernel_initializer='he_uniform'))
# Adding a dropout layer
model2.add(BatchNormalization())
model2.add(Dropout(0.4))
model2.add(LSTM(64, return_sequences=True))
model2.add(Conv1D(32, kernel_size=(3), activation='relu'))
model2.add(Dropout(0.5))
model2.add(Conv1D(64, kernel_size=(3) , activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(Flatten())
model2.add(Dense(40, activation='relu'))

model2.add(Dense(6, activation='softmax'))

In [0]:
model2.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

from keras.callbacks import ModelCheckpoint

filepath="/content/drive/My Drive/Colab Notebooks/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [92]:

history=model2.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=epochs, callbacks=callbacks_list, verbose=1)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 230s 31ms/step - loss: 0.3482 - acc: 0.8594 - val_loss: 0.3936 - val_acc: 0.8537

Epoch 00001: val_acc improved from -inf to 0.85375, saving model to /content/drive/My Drive/Colab Notebooks/weights-improvement-01-0.85.hdf5
Epoch 2/30
7352/7352 [==============================] - 230s 31ms/step - loss: 0.1437 - acc: 0.9407 - val_loss: 0.2065 - val_acc: 0.9253

Epoch 00002: val_acc improved from 0.85375 to 0.92535, saving model to /content/drive/My Drive/Colab Notebooks/weights-improvement-02-0.93.hdf5
Epoch 3/30
7352/7352 [==============================] - 231s 31ms/step - loss: 0.1486 - acc: 0.9404 - val_loss: 0.2495 - val_acc: 0.8975

Epoch 00003: val_acc did not improve from 0.92535
Epoch 4/30
7352/7352 [==============================] - 235s 32ms/step - loss: 0.1208 - acc: 0.9476 - val_loss: 0.1758 - val_acc: 0.9345

Epoch 00004: val_acc improved from 0.92535 to 0.93451, saving mod

**Model 3**

In [31]:
from keras.layers import BatchNormalization
# Initiliazing the sequential model
model3 = Sequential()
# Configuring the parameters
model3.add(LSTM(128, return_sequences=True, input_shape=(timesteps, input_dim), kernel_initializer='glorot_normal'))
# Adding a dropout layer
model3.add(Dropout(0.2))
model3.add(BatchNormalization())
model3.add(LSTM(64, return_sequences=True))
# Adding a dropout layer
model3.add(Dropout(0.5))
model3.add(Conv1D(32, kernel_size=(7), activation='relu'))
model3.add(Conv1D(24, kernel_size=(3) , activation='relu'))
model3.add(Dropout(0.6))
model3.add(MaxPooling1D(pool_size=3))
model3.add(Flatten())
model3.add(Dense(64, activation='relu'))
model3.add(Dense(6, activation='softmax'))


In [0]:
model3.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

from keras.callbacks import ModelCheckpoint

filepath="/content/drive/My Drive/Colab Notebooks/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [33]:
history=model3.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=epochs, callbacks=callbacks_list, verbose=1)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 252s 34ms/step - loss: 0.4155 - acc: 0.8377 - val_loss: 0.2822 - val_acc: 0.8948

Epoch 00001: val_acc improved from -inf to 0.89481, saving model to /content/drive/My Drive/Colab Notebooks/weights-improvement-01-0.89.hdf5
Epoch 2/30
7352/7352 [==============================] - 248s 34ms/step - loss: 0.1709 - acc: 0.9329 - val_loss: 0.2155 - val_acc: 0.9125

Epoch 00002: val_acc improved from 0.89481 to 0.91245, saving model to /content/drive/My Drive/Colab Notebooks/weights-improvement-02-0.91.hdf5
Epoch 3/30
7352/7352 [==============================] - 254s 35ms/step - loss: 0.1585 - acc: 0.9382 - val_loss: 0.2422 - val_acc: 0.9050

Epoch 00003: val_acc did not improve from 0.91245
Epoch 4/30
7352/7352 [==============================] - 249s 34ms/step - loss: 0.1379 - acc: 0.9419 - val_loss: 0.2042 - val_acc: 0.9131

Epoch 00004: val_acc improved from 0.91245 to 0.91313, saving mod

**BEST PERFORMANCE IN MODEL 3**

In [0]:
model3.load_weights("/content/drive/My Drive/Colab Notebooks/weights-improvement-24-0.94.hdf5")
model3.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
scores = model3.evaluate(X_test, Y_test, verbose=0)

In [35]:
print("%s: %.2f%%" % (model3.metrics_names[1], scores[1]*100))

acc: 94.27%


In [37]:
from prettytable import PrettyTable
    
x = PrettyTable()

x.field_names = ["Model Name","Number of LSTM Layers","Number of CNN Layers", "Dropout Implemented", "Batch Normalization implemented","Epochs Run","Initializer used", "Accuracy Achieved"]

x.add_row(["Model 1", 2, 3,"Yes", "Yes",30,"He Uniform", 0.938 ])
x.add_row(["Model 2", 2, 3,"Yes", "Yes",30,"He Uniform", 0.934 ])
x.add_row(["Model 3", 2, 3,"Yes", "Yes",30,"Glorot Normal", 0.942 ])
print(x)

+------------+-----------------------+----------------------+---------------------+---------------------------------+------------+------------------+-------------------+
| Model Name | Number of LSTM Layers | Number of CNN Layers | Dropout Implemented | Batch Normalization implemented | Epochs Run | Initializer used | Accuracy Achieved |
+------------+-----------------------+----------------------+---------------------+---------------------------------+------------+------------------+-------------------+
|  Model 1   |           2           |          3           |         Yes         |               Yes               |     30     |    He Uniform    |       0.938       |
|  Model 2   |           2           |          3           |         Yes         |               Yes               |     30     |    He Uniform    |       0.934       |
|  Model 3   |           2           |          3           |         Yes         |               Yes               |     30     |  Glorot Normal   | 

1) We used a combination of LSTM and CNN with softmax layers.
2) We trained 3 different models with different layer parameters, dropouts and initializers.
3) Model 3 performed the best by giving us an accuracy of 94.27%